# Load Data

In [1]:
import h5py
import numpy as np

In [2]:
data_path = "../data/robot_demonstrations/RoboTurkPilot/pegs-full"

# f = h5py.File("../data/robot_demonstrations/RoboTurkPilot/pegs-full", "r")

In [6]:
import collections
import json
import hashlib
import torch
from torch.utils.data import Dataset
import robosuite as rs
from robosuite.environments.base import MujocoEnv
import os


def xml_md5(xml_path: str) -> str:
    with open(xml_path, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()

class EnvCache:
    """
    default=True  → одна среда (экономно).
    default=False → LRU-кэш по md5(xml); max_envs ограничивает память.
    Параметр render включает интерактивное окно (debug).
    """
    def __init__(self,
                 task_name: str,
                 models_dir: str | None = None,
                 cam_name: str = "agentview",
                 img_size: int = 128,
                 default: bool = True,
                 max_envs: int | None = None,
                 render: bool = False):

        self.task_name  = task_name
        self.models_dir = models_dir
        self.cam_name   = cam_name
        self.img_size   = img_size
        self.default    = default
        self.max_envs   = max_envs or 32
        self.render     = render

        self._single_env: MujocoEnv | None = None
        self._lru: "collections.OrderedDict[str, MujocoEnv]" = collections.OrderedDict()
        self._has_custom_xml = "mujoco_model_path" in rs.environments.base.make.__code__.co_varnames

    # ---------- создание среды -------------
    def _create_env(self, xml_path: str | None = None) -> MujocoEnv:
        kwargs = dict(
            env_name              = self.task_name,
            robots                = "Sawyer",
            has_renderer          = self.render,
            has_offscreen_renderer= True,
            use_camera_obs        = True,
            camera_names          = [self.cam_name],
            camera_heights        = self.img_size,
            camera_widths         = self.img_size,
        )
        if xml_path and self._has_custom_xml:
            kwargs["mujoco_model_path"] = xml_path
        return rs.make(**kwargs)

    # ---------- основной API ---------------
    def get_env(self, xml_file: str | None = None) -> MujocoEnv:
        if self.default or not xml_file:
            if self._single_env is None:
                self._single_env = self._create_env()
            return self._single_env

        xml_path = os.path.join(self.models_dir, xml_file)
        key = xml_md5(xml_path)

        if key in self._lru:
            self._lru.move_to_end(key)
            return self._lru[key]

        env = self._create_env(xml_path)
        self._lru[key] = env
        self._lru.move_to_end(key)

        if len(self._lru) > self.max_envs:
            _, old_env = self._lru.popitem(last=False)
            old_env.close()
        return env


# ---------- dataset -----------------------------------------------------------
class SawyerDataset(Dataset):
    """
    PyTorch dataset that returns:
    All indices are pre‑computed once in create_trajectory_indices().
    """

    def __init__(self, data_path, horizon_left=2, horizon_right=8,
                 image_size=128, camera_name="agentview",
                 img_batch=2048, limit_demo=None,):

        self.data_path = data_path
        self.image_size = image_size
        self.camera_name = camera_name
        self.device = torch.device("cuda")

        f = h5py.File(f"{self.data_path}/demo.hdf5", "r")

        data_grp = f["data"]

        self.task_name = data_grp.attrs["env"]
        self.task_name = self.task_name.replace("Sawyer", "")

        demos = list(data_grp.keys())

        all_idx = []
        all_states = []
        all_joint_velocities = []
        all_gripper_actuation = []
        xml_per_state = []
        episode_ends = [0]

        if limit_demo and len(demos) > limit_demo:
            demos = demos[:limit_demo]


        models_dir = os.path.join(self.data_path, "models")
        env_cache = EnvCache(self.task_name, models_dir, render=True)

        for demo in demos:
            demo_data = data_grp[demo]
            # get data

            states = demo_data["states"][:]
            gripper_actuation = demo_data["gripper_actuations"][:]
            joint_velocities = demo_data["joint_velocities"][:]
            xml = demo_data.attrs["model_file"]

            # right_dpos = demo_data["right_dpos"][:]
            # right_dquat = demo_data["right_dquat"][:]

            # states logic
            # I work only with indices states[idx, :] so the only shape that is essential is the len of the trajectory
            traj_len, _ = states.shape

            window_template = np.arange(-horizon_left, horizon_right + 1)  # [W,]

            base = np.arange(0, traj_len - 1)[:, None]

            windows = base + window_template

            np.clip(windows, 0, traj_len, out=windows)

            windows = windows + episode_ends[-1]

            all_idx.append(windows)
            all_states.append(states)
            all_joint_velocities.append(joint_velocities)
            all_gripper_actuation.append(gripper_actuation)
            xml_per_state.extend([xml]*traj_len)

            episode_ends.append(episode_ends[-1] + traj_len)

        # np_all_idx = np.ndarray(all_idx)
        # np_all_states = np.ndarray(all_states)

        self.all_idx = np.concatenate(all_idx, axis=0)
        self.all_states = np.concatenate(all_states, axis=0)
        self.all_gripper_actuation = np.concatenate(all_gripper_actuation, axis=0)
        self.all_joint_velocities = np.concatenate(all_joint_velocities, axis=0)
        self.xml_per_state = np.array(xml_per_state)

        self.env = None

        self.img_dir = os.path.join(data_path, f"images_{camera_name}_{image_size}")
        self.img_bin = os.path.join(self.img_dir, "images.dat")
        self.meta_path = os.path.join(self.img_dir, "images.meta")

        os.makedirs(self.img_dir, exist_ok=True)

        if not (os.path.isfile(self.img_bin) and os.path.isfile(self.meta_path)):
            print("[SawyerDataset] rendering RGB frames ...")
            self._render_and_store(env_cache, batch=img_batch)

        # memmap для быстрого доступа
        with open(self.meta_path) as fp:
            meta = json.load(fp)                # {'N':..., 'H':..., 'W':...}
        N, H, W = meta["N"], meta["H"], meta["W"]
        self.img_mm = np.memmap(self.img_bin, mode='r',
                                dtype=np.uint8, shape=(N, H, W, 3))

    # ---------- рендер и сохранение ------------------------------------------
    def _render_and_store(self, env_cache: EnvCache, batch: int) -> None:
        N, H, W = len(self.all_states), self.image_size, self.image_size
        img_mm = np.memmap(self.img_bin, mode="w+", dtype=np.uint8,
                           shape=(N, H, W, 3))

        for start in range(0, N, batch):
            end = min(start + batch, N)
            for i in range(start, end):
                env = env_cache.get_env(self.xml_per_state[i])   # сцена
                env.reset()                                      # на всякий случай
                env.sim.set_state_from_flattened(self.all_states[i]) # qpos+qvel
                env.sim.forward()

                # В v1.5 off-screen рендер делается через sim.render
                bgr = env.sim.render(width=W, height=H,
                                     camera_name=self.camera_name)
                img_mm[i] = bgr[..., ::-1]                       # BGR → RGB
            print(f"rendered {end}/{N}", end="\r")

        img_mm.flush()
        with open(self.meta_path, "w") as fp:
            json.dump({"N": N, "H": H, "W": W}, fp)
        print(f"\n[SawyerDataset] images saved to {self.img_bin}")

    # ---------- инициализация memmap после сохранения ------------------------
    def _open_memmap(self):
        with open(self.meta_path) as fp:
            meta = json.load(fp)
        N, H, W = meta["N"], meta["H"], meta["W"]
        self.img_mm = np.memmap(self.img_bin, mode="r",
                                dtype=np.uint8, shape=(N, H, W, 3))

    # ---------- Dataset API ---------------------------------------------------
    def __len__(self):
        return len(self.all_idx)

    def __getitem__(self, i):
        window_idx = self.all_idx[i]                 # [W]
        center     = window_idx[len(window_idx)//2]

        state  = torch.tensor(self.all_states[center],
                              dtype=torch.float32)
        action = torch.tensor(
            np.concatenate([self.all_joint_velocities[center], self.all_gripper_actuation[center]], axis=-1),
            dtype=torch.float32
        )

        # берём RGB из memmap и превращаем в тензор [3,H,W]
        img = torch.from_numpy(self.img_mm[center])\
                  .permute(2,0,1).float() / 255.0

        return {"pixels": img.to(self.device),
                "state" : state.to(self.device),
                "action": action.to(self.device)}



    # # total number of windows
    # def __len__(self):
    #     return len(self.indexes)
    #
    # # slice arrays by pre‑computed row of indices
    # def __getitem__(self, idx):
    #     trajectory_idx = self.indexes[idx]
    #
    #     img_obs  = self.image_data_transformed[trajectory_idx[:self.obs_horizon + 1]]
    #     act_obs  = self.actions_data_transformed[trajectory_idx[:self.obs_horizon + 1]]
    #     act_pred = self.actions_data_transformed[trajectory_idx[self.obs_horizon + 1:]]
    #
    #     return {
    #         "img_obs" : img_obs,
    #         "act_obs" : act_obs,
    #         "act_pred" : act_pred,
    #     }


ds = SawyerDataset(data_path=data_path, horizon_left=2, horizon_right=8, limit_demo=5)

[SawyerDataset] rendering RGB frames ...


[robosuite INFO] Loading controller configuration from: /home/may33/projects/ml_portfolio/robotics/robosuite_src/robosuite/controllers/config/robots/default_sawyer.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/may33/projects/ml_portfolio/robotics/robosuite_src/robosuite/controllers/config/robots/default_sawyer.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/may33/projects/ml_portfolio/robotics/robosuite_src/robosuite/controllers/config/robots/default_sawyer.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/may33/projects/ml_portfolio/robotics/robosuite_src/robosuite/controllers/config/robots/default_sawyer.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/may33/projects/ml_portfolio/robotics/robosuite_src/robosuite/controllers/config/robots/default_sawyer.json (co

KeyboardInterrupt: 

In [11]:
# ─── зависимости ──────────────────────────────────────────
import os, json, hashlib, collections, h5py, numpy as np
import torch
from torch.utils.data import Dataset
from tqdm.auto import tqdm                    # ✓ прогресс-бар
import robosuite as rs
from robosuite.environments.base import MujocoEnv
# ───────────────────────────────────────────────────────────

def xml_md5(path: str) -> str:
    with open(path, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()

# ==========================================================
#                     ENV-КЭШ
# ==========================================================
class EnvCache:
    def __init__(self, task_name: str, models_dir: str | None,
                 cam_name="agentview", img_size=128,
                 default=True, max_envs=None, render=False):

        self.task   = task_name
        self.models = models_dir
        self.cam    = cam_name
        self.size   = img_size
        self.render = render
        self.deflt  = default
        self.maxN   = max_envs or 32

        self._one : MujocoEnv | None = None
        self._lru : "collections.OrderedDict[str, MujocoEnv]" = collections.OrderedDict()
        self._has_xml = "mujoco_model_path" in rs.environments.base.make.__code__.co_varnames

    def _new_env(self, xml: str | None = None) -> MujocoEnv:
        kw = dict(
            env_name               = self.task,
            robots                 = "Sawyer",
            has_renderer           = self.render,
            has_offscreen_renderer = True,      # ← ВСЕГДА TRUE!
            use_camera_obs         = True,
            camera_names           = [self.cam],
            camera_heights         = self.size,
            camera_widths          = self.size,
        )
        if xml and self._has_xml:
            kw["mujoco_model_path"] = xml
        return rs.make(**kw)

    def get_env(self, xml_file: str | None = None) -> MujocoEnv:
        if self.deflt or not xml_file:
            if self._one is None:
                self._one = self._new_env()
            return self._one

        path = os.path.join(self.models, xml_file)
        key  = xml_md5(path)
        if key in self._lru:                       # hit
            self._lru.move_to_end(key)
            return self._lru[key]

        env = self._new_env(path)                  # miss → создать
        self._lru[key] = env; self._lru.move_to_end(key)
        if len(self._lru) > self.maxN:             # LRU-ограничение
            _, old = self._lru.popitem(last=False)
            old.close()
        return env

# ==========================================================
#                  DATASET + РЕНДЕР
# ==========================================================
class SawyerDataset(Dataset):
    def __init__(self, data_path, horizon_left=2, horizon_right=8,
                 image_size=128, camera_name="agentview",
                 img_batch=4096, limit_demo=None):

        self.data_path = data_path
        self.img_size   = image_size
        self.camera     = camera_name
        self.device     = torch.device("cuda")

        f = h5py.File(os.path.join(data_path, "demo.hdf5"), "r")
        grp = f["data"]
        self.task = grp.attrs["env"].replace("Sawyer", "")
        demos = list(grp.keys())[:limit_demo] if limit_demo else list(grp.keys())

        # ── собираем все массивы ────────────────────────────
        idx, states, vel, grip, xmls, ends = [], [], [], [], [], [0]
        for d in demos:
            g  = grp[d]
            st = g["states"][:]
            n  = len(st)
            win = np.clip(np.arange(n-1)[:,None] + np.arange(-horizon_left, horizon_right+1), 0, n-1)
            idx.append(win + ends[-1])
            states.append(st)
            vel.append(g["joint_velocities"][:])
            grip.append(g["gripper_actuations"][:])
            xmls.extend([g.attrs["model_file"]]*n)
            ends.append(ends[-1]+n)

        self.idx   = np.concatenate(idx)
        self.state = np.concatenate(states)
        self.vel   = np.concatenate(vel)
        self.grip  = np.concatenate(grip)
        self.xmls  = np.array(xmls)

        # ── off-screen кэш ─────────────────────────────────
        self.ecache = EnvCache(self.task,
                               models_dir=os.path.join(data_path,"models"),
                               cam_name=self.camera,
                               img_size=image_size,
                               default=True,        # можно False, если нужен XML
                               render=False)

        # ── memmap для RGB ────────────────────────────────
        self.img_dir  = os.path.join(data_path, f"images_{camera_name}_{image_size}")
        self.img_bin  = os.path.join(self.img_dir, "images.dat")
        self.meta_js  = os.path.join(self.img_dir, "images.meta")
        os.makedirs(self.img_dir, exist_ok=True)

        if not (os.path.isfile(self.img_bin) and os.path.isfile(self.meta_js)):
            self._render_and_store(batch=img_batch)

        with open(self.meta_js) as fp:
            meta = json.load(fp)
        N,H,W = meta["N"], meta["H"], meta["W"]
        self.img_mm = np.memmap(self.img_bin, mode="r", dtype=np.uint8,
                                shape=(N,H,W,3))

    # ───────────────────────────────────────────────────────
    def _render_and_store(self, batch: int):
        N,H,W = len(self.state), self.img_size, self.img_size
        img_mm = np.memmap(self.img_bin, mode="w+", dtype=np.uint8,
                           shape=(N,H,W,3))

        for start in tqdm(range(0, N, batch), desc="render", unit="img"):
            for i in range(start, min(start+batch, N)):
                env = self.ecache.get_env(self.xmls[i])
                env.sim.set_state_from_flattened(self.state[i])
                env.sim.forward()
                bgr = env.sim.render(width=W, height=H, camera_name=self.camera)
                img_mm[i] = bgr[..., ::-1]            # BGR→RGB

        img_mm.flush()
        json.dump({"N":N,"H":H,"W":W}, open(self.meta_js,"w"))
        print(f"RGB-кадры сохранены в {self.img_bin}")

    # ── pytorch API ────────────────────────────────────────
    def __len__(self):  return len(self.idx)

    def __getitem__(self, i):
        w  = self.idx[i]
        c  = w[len(w)//2]
        img = torch.from_numpy(self.img_mm[c]).permute(2,0,1).float()/255.
        state  = torch.tensor(self.state[c], dtype=torch.float32)
        action = torch.tensor(np.concatenate([self.vel[c], self.grip[c]]),
                              dtype=torch.float32)
        return {"pixels": img.to(self.device),
                "state" : state.to(self.device),
                "action": action.to(self.device)}


In [12]:
ds = SawyerDataset(data_path=data_path,
                   horizon_left=2, horizon_right=8,
                   limit_demo=5)           # первый запуск — появится tqdm
sample = ds[0]
print(sample["pixels"].shape, sample["state"].shape, sample["action"].shape)

torch.Size([3, 128, 128]) torch.Size([47]) torch.Size([8])


In [13]:
import time

play_cache = EnvCache(task_name=ds.task,
                      models_dir=f"{ds.data_path}/models",
                      cam_name="agentview",
                      img_size=512,                # покрупнее
                      default=True,                # одна сцена достаточно
                      render=True)                 # <<< окно On

env = play_cache.get_env()                         # создаём среду + окно

# 3. Функция воспроизведения одной траектории
def play_traj(traj_idx: int, fps: int = 20):
    # берём все индексы кадров для этого демо
    # (вы строили self.idx так, что каждая демо идёт подряд)
    mask = ds.xmls == ds.xmls[ds.idx[traj_idx][0]]
    states = ds.state[mask]

    dt = 1.0 / fps
    for s in tqdm(states, desc=f"demo {traj_idx}"):
        env.sim.set_state_from_flattened(s)
        env.sim.forward()
        env.render()                               # показывает окно
        time.sleep(dt)

# 4. Проигрываем первые 3 демонстрации
for k in range(3):
    play_traj(k)

[robosuite INFO] Loading controller configuration from: /home/may33/projects/ml_portfolio/robotics/robosuite_src/robosuite/controllers/config/robots/default_sawyer.json (composite_controller_factory.py:121)
demo 0:  15%|█▌        | 978/6505 [00:49<04:39, 19.81it/s]


KeyboardInterrupt: 